In [26]:
## This code is without the str parser
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate

load_dotenv()

model = ChatOpenAI()

# 1st prompt
template1 = PromptTemplate(
    template= "Write a detailed report on {topic}",
    input_variables=["topic"],
)

# 2nd prompt
template2 = PromptTemplate(
    template= "Write a 5 line summary on the following text.\n{text}",
    input_variables=["text"],
)

prompt1 = template1.invoke('topic: AI in healthcare')
result = model.invoke(prompt1)

prompt2 = template2.invoke(f'text: {result.content}')
result2 = model.invoke(prompt2)

print(result2.content)

The use of Artificial Intelligence (AI) in healthcare shows promise in revolutionizing diagnostics, personalized medicine, and drug discovery. However, challenges such as data privacy, ethical considerations, and system integration must be addressed for successful adoption. Healthcare providers can benefit from AI technologies like machine learning, natural language processing, and computer vision to deliver more personalized and effective care to patients. AI algorithms can analyze medical imaging data, predict treatment responses, monitor patient health remotely, and identify potential drug candidates with efficiency and accuracy. With careful consideration of challenges, healthcare providers can harness AI's potential to improve patient outcomes and reduce costs.


### str output parser

In [32]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

model = ChatOpenAI()

# 1st prompt
template1 = PromptTemplate(
    template= "Write a detailed report on {topic}",
    input_variables=["topic"],
)

# 2nd prompt
template2 = PromptTemplate(
    template= "Write a 5 line summary on the following text.\n{text}",
    input_variables=["text"],
)

parser = StrOutputParser()

chain = template1 | model | parser | template2 | model | parser
result = chain.invoke('topic: AI in healthcare')
print(result)

Artificial intelligence (AI) is transforming healthcare by improving patient care, diagnostics, treatment planning, and operational efficiency. AI applications include disease detection, treatment planning, predictive analytics, virtual health assistants, and drug discovery. Benefits of AI in healthcare include improved patient outcomes, increased efficiency, cost savings, and enhanced research capabilities. Challenges include data privacy, ethical considerations, regulatory barriers, and resistance to change. Collaboration among healthcare stakeholders is essential to maximize the benefits of AI while addressing challenges and ensuring responsible use.


### JSON output parser

In [54]:
import os
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

load_dotenv()
#load hugginface api
huggingface_api_key = os.getenv("HUGGINGFACE_API_KEY")
huggingface_api_url = os.getenv("HUGGINGFACE_API_URL")

llm = HuggingFaceEndpoint(
    repo_id="modularai/Llama-3.1-8B-Instruct-GGUF",
    task="text2text-generation",
    huggingface_api_key=huggingface_api_key,
)
model = ChatHuggingFace(llm=llm)

parser = JsonOutputParser()

template1 = PromptTemplate(
    template= "Give me the name, age and city of a ficitional person \n {format_instructions}",
    input_variables=[],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

prompt = template1.format()
# print(prompt)

result = model.invoke(prompt)
final_result = parser.parse(result.content)
print(final_result)


WARNING! huggingface_api_key is not default parameter.
                    huggingface_api_key was transferred to model_kwargs.
                    Please make sure that huggingface_api_key is what you intended.


StopIteration: 

In [60]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

load_dotenv()
#load hugginface api
# huggingface_api_key = os.getenv("HUGGINGFACE_API_KEY")
# huggingface_api_url = os.getenv("HUGGINGFACE_API_URL")

# llm = HuggingFaceEndpoint(
#     repo_id="modularai/Llama-3.1-8B-Instruct-GGUF",
#     task="text2text-generation",
#     huggingface_api_key=huggingface_api_key,
# )
model = ChatOpenAI()

parser = JsonOutputParser()

template1 = PromptTemplate(
    template= "Give me the name, age and city of a ficitional person \n {format_instructions}",
    input_variables=[],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

prompt = template1.format()
# print(prompt)

# result = model.invoke(prompt)
# final_result = parser.parse(result.content)
# print(final_result)

chain = template1 | model | parser
result = chain.invoke({})
print(result)

## It does not flow the scheme for the data

{'name': 'Samantha Jones', 'age': 32, 'city': 'Springfield'}


### Structured output parser

In [69]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema


load_dotenv()

model = ChatOpenAI()

scheme = [
    ResponseSchema(
        name="fact1",
        description="fact 1 about the topic",
    ),
    ResponseSchema(
        name="fact2",
        description="fact 2 about the topic",
    ),
    ResponseSchema(
        name="fact2",
        description="fact 3 about the topic",
    ),
]

parser = StructuredOutputParser.from_response_schemas(scheme)

template_stuct = PromptTemplate(
    template="give me 3 facts about {topic} \n {format_instructions}",
    input_variables=["topic"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)


prompt = template_stuct.invoke('topic: Mars planet')
result = model.invoke(prompt)
final_result = parser.parse(result.content)

print(final_result)

{'fact1': 'Mars is the fourth planet from the Sun in our solar system.', 'fact2': 'Mars has the largest volcano in the solar system called Olympus Mons.', 'fact3': 'Evidence suggests that Mars may have had liquid water in the past, indicating the possibility of ancient life.'}


### Pydantic 

In [73]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

load_dotenv()

model = ChatOpenAI()

class person(BaseModel):
    name: str = Field(description="name of the person")
    age: int = Field(gt = 18, description="age of the person")
    city: str = Field(description="city of the person")

parser = PydanticOutputParser(pydantic_object=person)
template1 = PromptTemplate(
    template= "Give me the name, age and city of a ficitional person \n {format_instructions}",
    input_variables=[],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

prompt = template1.invoke({})
print(prompt)
result = model.invoke(prompt)
final_result = parser.parse(result.content)
print(final_result)


text='Give me the name, age and city of a ficitional person \n The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"description": "name of the person", "title": "Name", "type": "string"}, "age": {"description": "age of the person", "exclusiveMinimum": 18, "title": "Age", "type": "integer"}, "city": {"description": "city of the person", "title": "City", "type": "string"}}, "required": ["name", "age", "city"]}\n```'
name='Alice' age=25 city='New York'
